In [ ]:
os.path.getsize('C://Users//JCCLI//Downloads//used_cars_data.csv')

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql import SQLContext

In [4]:
import os
os.environ["HADOOP_HOME"] = "C:\Hadoop"
os.environ["JAVA_HOME"] ="C:\Program Files\Java\jdk1.8.0_341"

In [5]:
if __name__ == '__main__':
    scSpark = SparkSession \
        .builder \
        .appName("reading csv") \
        .config('spark.driver.extraClassPath', 'C:/spark-3.3.0-bin-hadoop3/jars/sqljdbc42.jar')\
        .config('spark.executor.extraClassPath', 'C:/spark-3.3.0-bin-hadoop3/jars/sqljdbc42.jar')\
        .getOrCreate()

In [ ]:
sqlsUrl = 'jdbc:sqlserver://localhost:1433;database=YourSQLDB'

In [6]:
data_file = 'C://Users//JCCLI//Downloads//used_carsnew.csv'
sdfData = scSpark.read.csv(data_file, header=True, sep=",").cache()

In [7]:
sdfData.registerTempTable("cars")

C:\Users\JCCLI\Miniconda_3\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [8]:
output =  scSpark.sql(
    "SELECT cars.height, cars.body_type, cars.year, cars.daysonmarket, cars.make_name, \
     cars.model_name, cars.has_accidents, cars.city, cars.seller_rating from cars")
output.show()

+-------+---------------+----+------------+----------+------------------+-------------+--------+-------------+
| height|      body_type|year|daysonmarket| make_name|        model_name|has_accidents|    city|seller_rating|
+-------+---------------+----+------------+----------+------------------+-------------+--------+-------------+
|66.5 in|SUV / Crossover|2019|         522|      Jeep|          Renegade|         null| Bayamon|          2.8|
|  68 in|SUV / Crossover|2020|         207|Land Rover|   Discovery Sport|         null|San Juan|          3.0|
|58.1 in|          Sedan|2016|        1233|    Subaru|           WRX STI|        False|Guaynabo|         null|
|  73 in|SUV / Crossover|2020|         196|Land Rover|         Discovery|         null|San Juan|          3.0|
|  68 in|SUV / Crossover|2020|         137|Land Rover|   Discovery Sport|         null|San Juan|          3.0|
|66.3 in|SUV / Crossover|2020|         242|Land Rover| Range Rover Velar|        False|San Juan|          3.0|
|

In [9]:
# Connnect to MS SQL
dest_tbl = "cars_table"
database = "usedcars"
user = "DESKTOP-CODJKMC\JCCLI"
password = "cars"

In [12]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [13]:
import psycopg2

In [15]:
conn = psycopg2.connect(
        host = "localhost",
        database = "usedcars",
        user = "postgres",
        password = "postgres")

In [16]:
cur = conn.cursor()

In [17]:
def create_table(cursor):
    
    cursor.execute("CREATE TABLE IF NOT EXISTS cars_table \
    (  height VARCHAR(12) NULL, \
       body_type VARCHAR(30) NULL, \
       year integer NULL, \
       daysonmarket integer(20) NULL, \
       make_name VARCHAR(50) NULL, \
       model_name VARCHAR(50) NULL, \
       has_accidents VARCHAR(10) NULL, \
       city VARCHAR(15) NULL, \
       seller_rating DECIMAL(3,1) NULL);")

In [21]:
def write_postgresql(output):
    cars_seq = [tuple(x) for x in output.collect()]
    records_list_template = ','.join(['%s'] * len(cars_seq))
    insert_query = "INSERT INTO cars_table (height, body_type, year, daysonmarket, make_name, model_name, \
                        has_accidents, city, seller_rating) VALUES {}".format(records_list_template)
    return insert_query, cars_seq

In [22]:
cur.execute(insert_query, cars_seq)

NameError: name 'insert_query' is not defined

In [ ]:
insert_query, cars_seq = write_postgresql(output)